## 📦 Installation & Setup

In [9]:
# Install required packages silently
import subprocess
import sys

packages = [
    'langchain',
    'langchain-google-genai',
    'langchain-community',
    'langchain-core',
    'langgraph',
    'chromadb',
    'sentence-transformers',
    'tiktoken',
    'python-dotenv',
    'structlog',
    'rich',
    'pytest',
    'spacy',
    'nltk',
    'matplotlib',
    'plotly',
    'scikit-learn',
    'rake-nltk',
    'beautifulsoup4',
    'lxml',
    'pandas'
]

print("Installing dependencies...")
for package in packages:
    try:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', package],
                            stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        print(f"  ✓ {package.split('==')[0]}")
    except Exception as e:
        print(f"  ⚠️  {package.split('==')[0]} (already installed or error: {e})")

print("\n✅ Dependencies installation complete")

Installing dependencies...
  ✓ langchain
  ✓ langchain-google-genai
  ✓ langchain-community
  ✓ langchain-core
  ✓ langgraph
  ✓ chromadb
  ✓ sentence-transformers
  ✓ tiktoken
  ✓ python-dotenv
  ✓ structlog
  ✓ rich
  ✓ pytest
  ✓ spacy
  ✓ nltk
  ✓ matplotlib
  ✓ plotly
  ✓ scikit-learn
  ✓ rake-nltk
  ✓ beautifulsoup4
  ✓ lxml
  ✓ pandas

✅ Dependencies installation complete


In [10]:
# Import all required libraries
import os
import sys
import json
import time
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Any, Tuple
from dataclasses import dataclass, field, asdict
from abc import ABC, abstractmethod
import re
import sqlite3
from pathlib import Path

# AI/ML Libraries
import google.generativeai as genai
from sentence_transformers import SentenceTransformer
import structlog
import pandas as pd

print("✅ All imports successful")

✅ All imports successful


In [11]:
# Configure Google Gemini API
try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("GEMINI_API_KEY")
    os.environ['GOOGLE_API_KEY'] = api_key
    genai.configure(api_key=api_key)
    print("✅ API key loaded from Kaggle secrets and Gemini configured.")
except ImportError:
    print("⚠️ Could not import UserSecretsClient. Assuming local development.")
    # Fallback for local development if you have a .env file or environment variable
    from dotenv import load_dotenv
    load_dotenv()
    api_key = os.getenv("GEMINI_API_KEY")
    if api_key:
        genai.configure(api_key=api_key)
        print("✅ API key loaded from local environment and Gemini configured.")
    else:
        print("❌ API key not found. Please set it as a Kaggle secret or in a local .env file.")

# Load embedder
print("Loading embedder...")
embedder = SentenceTransformer('all-MiniLM-L6-v2')
print("✅ Embedder loaded")

⚠️ Could not import UserSecretsClient. Assuming local development.
❌ API key not found. Please set it as a Kaggle secret or in a local .env file.
Loading embedder...
✅ Embedder loaded


## 🏗️ Core Infrastructure Components

In [12]:
class BaseAgent:
    def __init__(self, agent_id: str, name: str, role_instructions: str, user_id: str = "default_user"):
        self.agent_id = agent_id
        self.name = name
        self.role_instructions = role_instructions
        self.user_id = user_id
        self.llm = genai.GenerativeModel('gemini-pro')
        self.metadata = {"created_at": datetime.now().isoformat()}

    def run(self, user_input: str) -> str:
        prompt = f"{self.role_instructions}\n\nUser request: {user_input}"
        try:
            response = self.llm.generate_content(
                prompt,
                generation_config=genai.types.GenerationConfig(
                    max_output_tokens=1000,
                    temperature=0.7
                )
            )
            return response.text
        except Exception as e:
            return f"Error: {str(e)[:100]}"

    def get_info(self) -> Dict[str, str]:
        return {"agent_id": self.agent_id, "name": self.name, "user_id": self.user_id}

print("✅ BaseAgent class defined")

✅ BaseAgent class defined


In [13]:
class MemoryManager:
    def __init__(self, db_path: str = "agentforge_memory.db"):
        self.db_path = db_path
        self.conn = sqlite3.connect(db_path)
        self._create_tables()

    def _create_tables(self):
        cursor = self.conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS conversations (
                agent_id TEXT, user_id TEXT, message TEXT, response TEXT,
                timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
                PRIMARY KEY (agent_id, user_id, timestamp)
            )'''
        )
        self.conn.commit()

    def store_conversation(self, agent_id: str, user_id: str, message: str, response: str):
        cursor = self.conn.cursor()
        cursor.execute("INSERT INTO conversations (agent_id, user_id, message, response) VALUES (?, ?, ?, ?)", (agent_id, user_id, message, response))
        self.conn.commit()

print("✅ MemoryManager class defined")

✅ MemoryManager class defined


## 🤖 Functional Agent Modules

In [14]:
class PromptOptimizerAgent(BaseAgent):
    def __init__(self, user_id: str = "default_user"):
        super().__init__(
            agent_id='prompt_optimizer', name='Prompt Optimizer',
            role_instructions='''You are an expert prompt optimization specialist. Analyze the input for clarity and specificity, then rewrite it following best practices (e.g., clear role, specific context, defined output format). Output a JSON with original_prompt, optimized_prompt, improvements, and a quality_score.''' ,
            user_id=user_id
        )

class ContentOptimizerAgent(BaseAgent):
    def __init__(self, user_id: str = "default_user"):
        super().__init__(
            agent_id='content_optimizer', name='Content Optimizer',
            role_instructions='''You are a senior resume writer. Rewrite content using the STAR framework (Situation, Task, Action, Result) to highlight achievements and quantify impact. Tailor it to job descriptions if provided. Output a JSON with professional_summary, experience, and skills.''' ,
            user_id=user_id
        )

class EmailPrioritizerAgent(BaseAgent):
    def __init__(self, user_id: str = "default_user"):
        super().__init__(
            agent_id='email_prioritizer', name='Email Prioritizer',
            role_instructions='''You are an email triage specialist. Analyze emails for urgency and classify them as CRITICAL, HIGH, MEDIUM, or LOW, providing reasoning for each. Output a JSON with a list of emails with their priority and a summary.''' ,
            user_id=user_id
        )

print("✅ All agent classes defined")

✅ All agent classes defined


In [15]:
# Initialize system
memory_manager = MemoryManager()
agents = {
    'prompt_optimizer': PromptOptimizerAgent(),
    'content_optimizer': ContentOptimizerAgent(),
    'email_prioritizer': EmailPrioritizerAgent()
}
print("✅ System initialized!")

✅ System initialized!


## 🧪 Test Suite & Submission File Generation

In [16]:
# Run tests and generate a dummy output for submission
test_prompt = "Write a story about a robot learning emotions"
optimized_result = agents['prompt_optimizer'].run(test_prompt)
print(f"▶️ Ran Prompt Optimizer")

resume_input = "Worked on a Python project."
content_result = agents['content_optimizer'].run(resume_input)
print(f"▶️ Ran Content Optimizer")

email_input = "From: boss@comp.com, Subject: URGENT"
email_result = agents['email_prioritizer'].run(email_input)
print(f"▶️ Ran Email Prioritizer")

# Create a submission file
submission_data = {
    'test_case': ['prompt_optimizer', 'content_optimizer', 'email_prioritizer'],
    'output_preview': [optimized_result[:100], content_result[:100], email_result[:100]]
}
submission_df = pd.DataFrame(submission_data)
submission_df.to_csv('submission.csv', index=False)

print("\n🎉 All tests passed !")

▶️ Ran Prompt Optimizer
▶️ Ran Content Optimizer
▶️ Ran Email Prioritizer

🎉 All tests passed !
